The notebook outlines the telescope pointing routine to a LEO satellite from NORAD catalogue based on Two-Line Elements (TLE) data. The illustration below shows the baseline geometry of the mathematical model.

<!-- ![pointing_geometry.png](attachment:pointing_geometry.png) -->
<img src="pointing_geometry.png" width="500" height="340"> 

Two reference frame are depicted. Earth-centered Earth-fixed (ECEF) denoted as $\textit{OXYZ}$ is used to specify the location of the observatory on the Earth. Local reference frame $\textit{oxyz}$ originated at the observatory location, x-axis points toward North, z-axis points to zenith, y-axis completes orthonormal reference frame. The reference frame is used to specify azimuth and elevation of a satellite as seen from the observatory location.


$\mathbf{R}_{obs}$ represents observatory position vector given ECEF frame $\textit{OXYZ}$.

$\mathbf{R}_{sat}$ represents satellites position vector given ECEF frame $\textit{OXYZ}$.

In order to find azimuth and declination of a satellite for a particular time step the following procedure is used:

1. Propagate satellite state to obtain satellite position $\mathbf{R}_{sat}$ at time t. 
    Let's use SGP4 propagator that utilizes NORAD TLE as an input as a standard in the aerospace industry. The SGP4 outputs position and velocity given in ECI frame. It should be converted to ECEF frame where there observatory location is represented.
2. Find vector $\mathbf{r}$ pointing from the observatory to the satellite.
3. Find satellite azimuth $\alpha$ and declination $\beta$ as seen from the observatory by transforming $\mathbf{r}$ to local reference frame $\textit{oxyz}$ and then calculating the required angles as shown in illustration above.
   
Repeat the routine to obtain relative satellite trajectory arc, namely $\alpha(t)$, $\beta(t)$.

In [44]:
from consts import *

latitudeObs = 24 * np.pi / 180
longitudeObs = 54 * np.pi / 180

Robs = np.vstack([Consts.rEarth * np.cos(latitudeObs) * np.cos(longitudeObs), Consts.rEarth * np.cos(latitudeObs) * np.sin(longitudeObs), Consts.rEarth * np.sin(latitudeObs)])  

print(Robs)

[[3421031.44757223]
 [4708645.83380174]
 [2591322.81366571]]


In [38]:
from sgp4.api import Satrec
from datetime import datetime, timedelta
from astropy.time import Time
import numpy as np
import pandas as pd


s = "1 44714U 19074B   24303.16471469  .00014028  00000+0  95926-3 0  9999"
t = "2 44714  53.0541 255.6079 0001757  96.6875 263.4314 15.06381615274085"

satellite = Satrec.twoline2rv(s, t)


# Define start and end dates with specific times
start_date = "2024-10-29 13:00:00"
end_date = "2024-10-30 13:00:00"

# Generate a range of dates with hourly frequency
date_range = pd.date_range(start=start_date, end=end_date, freq='1min')

# Convert to a list of strings if needed
gregorian_dates = date_range.strftime("%Y-%m-%d %H:%M:%S").tolist()

# Convert to Julian Date
julian_dates = Time(gregorian_dates, format='iso').jd


# Propagate for each Julian Date
positionsX = []
positionsY = []
positionsZ = []

for jd in julian_dates:
    # Convert JD to days since epoch

    # Propagate
    e, r, v = satellite.sgp4(jd, 0)
    if e == 0:
        positionsX.append(r[0])
        positionsY.append(r[1])
        positionsZ.append(r[2])

    else:
        positionsX.append(None)  # Error in propagation
        positionsY.append(None)  # Error in propagation
        positionsY.append(None)  # Error in propagation


gregorian_dates = Time(julian_dates, format='jd').to_datetime()

sat1Data = np.array([gregorian_dates, positionsX, positionsY, positionsZ]).T

sat1 = pd.DataFrame(sat1Data, columns=["Date and Time", "X, km", "Y, km", "Z, km"])

print("Satellite 1 Position Data")
print(sat1)

Satellite 1 Position Data
                  Date and Time        X, km        Y, km        Z, km
0    2024-10-29 12:59:59.999987 -2848.596213  3811.088141 -5039.881683
1    2024-10-29 13:00:59.999983 -3065.798327  3436.980048 -5179.642172
2    2024-10-29 13:02:00.000020 -3269.825898  3048.101833 -5297.082709
3    2024-10-29 13:03:00.000016 -3459.804845  2646.127932 -5391.701877
4    2024-10-29 13:04:00.000013 -3634.921657  2232.787116 -5463.096139
...                         ...          ...          ...          ...
1436 2024-10-30 12:56:00.000001 -3225.575056  2939.556139 -5384.768845
1437 2024-10-30 12:56:59.999997 -3433.539255  2542.068709 -5458.071673
1438 2024-10-30 12:57:59.999994 -3626.755282  2133.662222 -5507.865513
1439 2024-10-30 12:58:59.999990 -3804.394715   1716.09182 -5533.938186
1440 2024-10-30 12:59:59.999987 -3965.695485  1291.151341  -5536.17869

[1441 rows x 4 columns]


In [12]:
print(julian_dates)

[2460613.04166667 2460613.04236111 2460613.04305556 ... 2460614.04027778
 2460614.04097222 2460614.04166667]


In [59]:
# Transforming satellite position to ECEF frame
julian_century = (julian_dates[0] - 2451545.0) / 36525.0
ecef_to_eci_rotation = rotation_matrix_ecef_to_eci(julian_century)

R_ECEF = np.matmul(ecef_to_eci_rotation, np.vstack([positionsX[0], positionsY[0], positionsZ[0]]))
print(R_ECEF)
print(ecef_to_eci_rotation)
print(julian_century)

[[ 4745.86445718]
 [  -15.95712251]
 [-5051.31829446]]
[[-6.01340420e-01  7.98989303e-01  2.40685089e-03]
 [-7.98991561e-01 -6.01342234e-01  3.83340121e-05]
 [ 1.47796956e-03 -1.90000176e-03  9.99997103e-01]]
0.248269450148296


In [57]:
import numpy as np
from calc_local_bases_axes import *

lat = 0
lon = 0
print(lat, lon)

A = calc_local_bases_axes(lat, lon)

print(A)
Rsat = np.vstack([positionsX[0], positionsY[0], positionsZ[0]])

r = R_ECEF - Robs

rTransformed = np.matmul(A, r)

az, decl = calc_local_azimuth_and_declination(rTransformed)

# print(r)

# print(rTransformed)

print(az * 180 / np.pi, decl * 180 / np.pi)

0 0
[[-0.  1.  0.]
 [-0. -0.  1.]
 [ 1.  0.  0.]]
[118.8725102] [-6.77000258e-06]


In [7]:
from datetime import datetime, timedelta

from conversions import *

# Convert UTC time to Julian Century
observation_time =  datetime(year=2021, month=3, day=19, hour=7, second=41)
julian_date = utc_time_to_julian_date(observation_time)
julian_century = (julian_date - 2451545.0) / 36525.0

# Compute the rotation matrix
ecef_to_eci_rotation = rotation_matrix_ecef_to_eci(julian_century)
print(ecef_to_eci_rotation)

[[ 2.07067703e-01  9.78324511e-01  2.02939438e-03]
 [-9.78326529e-01  2.07068110e-01  9.76248199e-06]
 [-4.10671983e-04 -1.98743186e-03  9.99997941e-01]]
